# Transfer Learning with TensorFlow Part 3: Sclaing up (Food Vision mini)

We've seen the power of transfer learning feature extraction and fine-tuning, now it's time to scale up to all of the classes in Food101 (101 total classes).

Goal is to beat the Food101 paper with 10% of the data.

In [1]:
# Check GPU
!nvidia-smi

Mon Jan 16 14:08:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.92.01    Driver Version: 528.02       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:04:00.0  On |                  N/A |
| 53%   40C    P8    51W / 390W |    841MiB / 24576MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Helper functions @ ./helper_functions.py

In [3]:
# Import series of helper functions from helper_functions.py
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

2023-01-16 14:08:53.872919: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 14:08:55.806382: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2023-01-16 14:08:55.806451: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2023-01-16 14:08:55.808410: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd3d12.so
2023-01-16 14:08:56.095591: I tensorflow/c/logging.cc:34] DirectML device enumeration: found 1 compatible adapters.


## 101 Food Classes : Working with less data

In [4]:
# Download, data has been modified using image_data_modification notebook on course github - https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/06_transfer_learning_in_tensorflow_part_3_scaling_up.ipynb
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
unzip_data("101_food_classes_10_percent.zip")

--2023-01-16 14:08:57--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.138.128, 142.251.32.144, 142.251.33.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.138.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip.1’

101_food_classes_10 100%[===================>]   1.51G  85.0MB/s    in 17s     

2023-01-16 14:09:14 (93.1 MB/s) - ‘101_food_classes_10_percent.zip.1’ saved [1625420029/1625420029]



In [5]:
unzip_data("101_food_classes_10_percent.zip")

In [6]:
# manually moving 101_food_classes_10_percent unzipped folder to the data folder

In [7]:
train_dir = "../data/101_food_classes_10_percent/train"
test_dir = "../data/101_food_classes_10_percent/test"

In [8]:
# how many images/classes are there
walk_through_dir("../data/101_food_classes_10_percent")

There are 2 directories and 0 images in '../data/101_food_classes_10_percent'.
There are 101 directories and 0 images in '../data/101_food_classes_10_percent/test'.
There are 0 directories and 250 images in '../data/101_food_classes_10_percent/test/macaroni_and_cheese'.
There are 0 directories and 250 images in '../data/101_food_classes_10_percent/test/cannoli'.
There are 0 directories and 250 images in '../data/101_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '../data/101_food_classes_10_percent/test/grilled_cheese_sandwich'.
There are 0 directories and 250 images in '../data/101_food_classes_10_percent/test/spring_rolls'.
There are 0 directories and 250 images in '../data/101_food_classes_10_percent/test/apple_pie'.
There are 0 directories and 250 images in '../data/101_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '../data/101_food_classes_10_percent/test/ceviche'.
There are 0 directories and 250 images 

In [9]:
# Set up data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all_10_percent= tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                               label_mode="categorical",
                                                                               image_size=IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                               shuffle=False) # don't shuffle test data for prediction analysis

Found 7575 files belonging to 101 classes.


2023-01-16 14:09:29.962462: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 14:09:29.965153: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 0 (NVIDIA GeForce RTX 3090)
2023-01-16 14:09:30.112591: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-16 14:09:30.112620: W tensorflow/core/common_runtime/pluggable_device/pluggable_device_bfc_allocator.cc:28] Overriding allow_growth setting because force_memory_growth was requested by the device.
2023-01-16 14:09:30.112636: I tensorflow/core/common_runtime/pluggable_device/

Found 25250 files belonging to 101 classes.


## Train a big dog model with transfer learning on 10% of the 101 food classes

Steps:
* Create a `ModelCheckpoint` callback
* Create a data augmentation layer to build data augmentation right into the model
* Build a headless (no top layers) Functional EfficientNetB0 backboned-model (we'll create our own output layer)
* Compile our model
* Feature extract for 5 epochs, and 15% of teh test data to save time
                                                                    

In [10]:
# Create a ModelCheckpoint
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                        save_weights_only=True,
                                                        monitor="val_accuracy",
                                                        save_best_only=True)

In [11]:
# Create data augmentation layer to incorporate it right into the model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data augmentation
data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2)
    # preprocessing.Rescaling(1/255.) # rescale inputs between 0 and 1, required for models like resnet 50
], name="data_augmentation")

In [12]:
# Setup the base model and freeze its layers (this will extract features)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Setup model architecture with trainable top layers
inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
x = data_augmentation(inputs) # augment images (only happens during the training phase)
x = base_model(x, training=False) # put the model in inference mode so weights stay frozen
x = layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation = "softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

In [13]:
# get a summary of model we created
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_avg_pool_layer (Glob  (None, 1280)             0         
 alAveragePooling2D)                                             
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-trainab

In [ ]:

with tf.device("/GPU:0"):
    # Compile
    model.compile(loss="categorical_crossentropy",
                  optimizer= tf.keras.optimizers.Adam(),
                  metrics=["accuracy"])

    # fit
    history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                               epochs=5,
                                               validation_data=test_data,
                                               validation_steps=int(0.15 * len(test_data)),
                                               callbacks=[checkpoint_callback])

Epoch 1/5


2023-01-16 14:23:49.796226: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-16 14:23:50.478406: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-16 14:23:50.478493: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 83006 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-01-16 14:23:50.478537: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-01-16 14:23:50.479081: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identi

237/237 [==============================] - ETA: 0s - loss: 1.8239 - accuracy: 0.5624

2023-01-16 14:30:55.854295: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-16 14:30:56.472977: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-16 14:30:56.473043: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 83006 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-01-16 14:30:56.473086: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-01-16 14:30:56.473771: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identi

237/237 [==============================] - 438s 2s/step - loss: 1.8239 - accuracy: 0.5624 - val_loss: 1.9252 - val_accuracy: 0.5061
Epoch 2/5
237/237 [==============================] - 666s 3s/step - loss: 1.5911 - accuracy: 0.6065 - val_loss: 1.8296 - val_accuracy: 0.5267
Epoch 3/5
237/237 [==============================] - 642s 3s/step - loss: 1.4361 - accuracy: 0.6455 - val_loss: 1.7445 - val_accuracy: 0.5434
Epoch 4/5
237/237 [==============================] - 431s 2s/step - loss: 1.3128 - accuracy: 0.6754 - val_loss: 1.7717 - val_accuracy: 0.5392
Epoch 5/5
189/237 [======================>.......] - ETA: 41s - loss: 1.2249 - accuracy: 0.6959

In [ ]:
fine_tuning_results = model.evaluate(test_data)

In [ ]:
plot_loss_curves(history_all_classes_10_percent)

## Fine-tuning

In [ ]:
# Unfreeze all of the layers in the base model
base_model.trainable = True

# Refreeze every layer except last 5
for layer in base_model.layers[:-5]:
    layer.trainable = False

In [ ]:
# recompile model with lower learning rate (lower by 10x)
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=.0001),
              metrics=["accuracy"])


In [ ]:
# what layers in the model are trainable
for layer in model.layers:
    print(layer.name, layer.trainable)

In [ ]:
# check which layers are trainable in base_model
for layer_number, layer in enumerate(model.layers[2].layers):
    print(layer_number, layer.name, layer.trainable)

In [ ]:
tf.debugging.set_log_device_placement(True)


# with tf.device('/GPU:0'): 
# Fine-tune for 5 more people
fine_tune_epochs = 10 # model has done 5 already, this is the total desired.

# fine-tune our model
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs=fine_tune_epochs,
                                           validation_data = test_data,
                                           validation_steps = int(0.15 * len(test_data)),
                                           initial_epoch=history_all_classes_10_percent.epoch[-1])